# 🌾 Рекомендация удобрений — исследование

Анализ данных, обучение моделей, симуляция урожайности.

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)
print('Готово')

## 1. Загрузка данных

In [ ]:
data_path = 'data/fertilizer_dataset.csv'
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print(f'Записей: {len(df)}')
    print(f'Культуры: {df["crop"].nunique()} — {sorted(df["crop"].unique())}')
    df.describe().round(2)
else:
    print('Данные не найдены. Запустите: python train.py')
    from train import generate_dataset
    df = generate_dataset(10000)
    df.to_csv(data_path, index=False)

In [ ]:
df.head(10)

## 2. Распределение признаков

In [ ]:
num_cols = ['temperature', 'humidity', 'rainfall', 'soil_ph', 
            'soil_N', 'soil_P', 'soil_K', 'prev_yield',
            'N_recommended', 'P_recommended', 'K_recommended']

fig, axes = plt.subplots(3, 4, figsize=(18, 12))
for ax, col in zip(axes.flatten(), num_cols):
    df[col].hist(bins=40, ax=ax, color='steelblue', edgecolor='white')
    ax.set_title(col, fontweight='bold')
    ax.grid(alpha=0.3)
axes.flatten()[-1].axis('off')
plt.suptitle('Распределение признаков', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('results/feature_distributions.png', dpi=150)
plt.show()

## 3. Рекомендации по культурам

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, nutrient, color in zip(axes, ['N_recommended', 'P_recommended', 'K_recommended'],
                                     ['#e74c3c', '#3498db', '#2ecc71']):
    crop_means = df.groupby('crop')[nutrient].mean().sort_values(ascending=True)
    crop_means.plot.barh(ax=ax, color=color, edgecolor='white')
    ax.set_title(nutrient.replace('_', ' ').title(), fontweight='bold')
    ax.set_xlabel('кг/га')
    ax.grid(axis='x', alpha=0.3)

plt.suptitle('Средние рекомендуемые дозы по культурам', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('results/npk_by_crop.png', dpi=150)
plt.show()

## 4. Корреляционная матрица

In [ ]:
corr_cols = ['temperature', 'humidity', 'rainfall', 'soil_ph', 'soil_N', 'soil_P', 'soil_K',
             'prev_yield', 'N_recommended', 'P_recommended', 'K_recommended']

fig, ax = plt.subplots(figsize=(12, 10))
corr = df[corr_cols].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, ax=ax, linewidths=0.5)
ax.set_title('Корреляционная матрица', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('results/correlation_matrix.png', dpi=150)
plt.show()

print('\nКлючевые корреляции с N_recommended:')
print(corr['N_recommended'].drop('N_recommended').sort_values().to_string())

## 5. Зависимость NPK от содержания в почве

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

pairs = [('soil_N', 'N_recommended'), ('soil_P', 'P_recommended'), ('soil_K', 'K_recommended')]
colors = ['#e74c3c', '#3498db', '#2ecc71']

for ax, (soil, rec), c in zip(axes, pairs, colors):
    ax.scatter(df[soil], df[rec], alpha=0.15, s=8, color=c)
    z = np.polyfit(df[soil], df[rec], 2)
    p = np.poly1d(z)
    x_line = np.linspace(df[soil].min(), df[soil].max(), 100)
    ax.plot(x_line, p(x_line), 'k-', linewidth=2, label='Тренд')
    ax.set_xlabel(f'{soil} (кг/га)')
    ax.set_ylabel(f'{rec} (кг/га)')
    ax.set_title(f'{soil} → {rec}', fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.suptitle('Чем больше в почве — тем меньше нужно добавлять', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('results/soil_vs_recommendation.png', dpi=150)
plt.show()

## 6. Кривая отклика Митчерлиха

In [ ]:
from train import CROP_PARAMS, mitscherlich_yield

fig, axes = plt.subplots(2, 4, figsize=(18, 8))

N_range = np.linspace(0, 300, 200)

for ax, (crop, p) in zip(axes.flatten(), CROP_PARAMS.items()):
    yields = [mitscherlich_yield(n, p['opt_P'], p['opt_K'], p,
                                 p['temp_opt'], p['ph_opt']) for n in N_range]
    ax.plot(N_range, yields, 'b-', linewidth=2)
    ax.axvline(p['opt_N'], color='red', linestyle='--', alpha=0.7, label=f'Opt N={p["opt_N"]}')
    ax.set_title(crop.capitalize(), fontweight='bold')
    ax.set_xlabel('N (кг/га)')
    ax.set_ylabel('Урожай (т/га)')
    ax.legend(fontsize=8)
    ax.grid(alpha=0.3)

plt.suptitle('Кривая отклика урожая на азот (Митчерлих)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('results/mitscherlich_curves.png', dpi=150)
plt.show()
print('Закон убывающей отдачи: каждый следующий кг удобрений даёт всё меньший прирост')

## 7. Результаты обучения

```bash
python train.py --samples 10000 --epochs 200
python evaluate.py
```

In [ ]:
if os.path.exists('results/full_report.json'):
    with open('results/full_report.json', encoding='utf-8') as f:
        report = json.load(f)
    
    metrics = report['model_metrics']
    print(f"Лучшая модель: {report['best_model']}\n")
    print(f"{'Модель':<15} {'N MAE':>7} {'P MAE':>7} {'K MAE':>7} {'Avg R²':>7} {'MAPE':>7}")
    print('-' * 55)
    for name, m in sorted(metrics.items(), key=lambda x: x[1].get('AVG', {}).get('MAPE', 100)):
        print(f"{name:<15} {m['N']['MAE']:>7.2f} {m['P']['MAE']:>7.2f} {m['K']['MAE']:>7.2f} "
              f"{m['AVG']['R2']:>7.4f} {m['AVG']['MAPE']:>6.1f}%")
    
    print(f"\nСимуляция урожайности:")
    sim = report['yield_simulation']
    for name, r in sim.items():
        print(f"  {name}: прирост vs baseline = {r['improvement_vs_baseline_%']:+.1f}%, "
              f"экономия удобрений = {r['cost_saving_%']:+.1f}%")
else:
    print('Запустите train.py и evaluate.py сначала.')

In [ ]:
# Отображение сохранённых графиков
plots = ['results/model_comparison.png', 'results/scatter_plots.png', 
         'results/yield_simulation.png', 'results/mlp_training.png']

for plot_path in plots:
    if os.path.exists(plot_path):
        from PIL import Image
        img = Image.open(plot_path)
        plt.figure(figsize=(14, 6))
        plt.imshow(img); plt.axis('off')
        plt.title(os.path.basename(plot_path), fontweight='bold')
        plt.show()

## 8. Пример рекомендации

In [ ]:
# Быстрый пример — рекомендация для пшеницы
import subprocess
result = subprocess.run([
    'python', 'predict.py',
    '--crop', 'wheat',
    '--temperature', '22',
    '--humidity', '55',
    '--rainfall', '100',
    '--soil_ph', '6.3',
    '--soil_N', '35',
    '--soil_P', '18',
    '--soil_K', '22',
    '--prev_yield', '3.8',
], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print(result.stderr)

## 9. Выводы

### Результаты:
1. **XGBoost / MLP** показывают лучшие результаты по MAE и R²
2. Модели корректно выучили **обратную зависимость**: больше в почве → меньше рекомендация
3. **Симуляция** подтверждает: рекомендации дают прирост урожая +10-25% vs фиксированные дозы
4. **Экономия** удобрений 10-30% при сохранении/увеличении урожая

### Модель Митчерлиха:
- Закон убывающей отдачи реалистично отражён
- Учтены: температура, pH, сезонность
- Модель служит и для генерации данных, и для валидации рекомендаций

### Возможные улучшения:
- Реальные полевые данные вместо синтетических
- Reinforcement Learning для оптимизации многолетних стратегий
- GNN для учёта пространственной структуры полей
- Добавление стоимости удобрений в целевую функцию